In [2]:
import torch
from dataloader import SeqComb
train_set = SeqComb.get_SeqComv('onetwo', 'TRAIN')
train_set.X.shape, train_set.y.shape

/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(torch.Size([5000, 100, 1]), torch.Size([5000]))

In [3]:


weights = torch.tensor([1/100]).repeat(64, 100)
sorted, _ = torch.multinomial(weights, 2, replacement=True).float().sort()
start, end = sorted[:, 0], sorted[:, 1]
(end-start).mean()


tensor(36.7031)

In [4]:
import ppo

In [5]:
import torch
pth = torch.load("./checkpoints/pred_onetwo_ppo_tcn_cat_nb_0.7,0.3/onetwo_ppo_tcn_cat_nb_0.7,0.3.pth")

<ipython-input-5-4e3a8062ab09>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pth = torch.load("./checkpoints/pred_onetwo_ppo_tcn_cat_nb_0.7,0.3/onetwo_ppo_tcn_cat_nb_0.7,

FileNotFoundError: [Errno 2] No such file or directory: './checkpoints/pred_onetwo_ppo_tcn_cat_nb_0.7,0.3/onetwo_ppo_tcn_cat_nb_0.7,0.3.pth'

In [ ]:
from utils.plot import result_plots

result_plots(
    loss='ppo',
    backbone='tcn',
    weights=[0.7, 0.3],
    seg_dist='cat_nb',
    dataset='onetwo'
)

onetwo_ppo_tcn_cat_nb_0.7,0.3


/home/hjkim/RL_TimeSegment/utils/plot.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"{exp_dir}/{exp_name}.pth")


In [ ]:
from models import Policy, Value

In [1]:
from models import Policy, Value
policy_module = Policy.init_policy_module(
    d_in = 1,
    d_model = 128,
    d_out = 3,
    seq_len = 100,
    backbone = 'tcn',
    seg_dist = 'cat_nb'
)
value_module, advantage_module = Value.init_value_module(
    d_in = 1,
    d_model = 128,
    seq_len = 100,
    backbone = 'tcn'
)

/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [6]:
next(policy_module.parameters())[0]

tensor(0.0795, grad_fn=<SelectBackward0>)

In [ ]:
from torchrl.objectives import ClipPPOLoss

loss_module = ClipPPOLoss(
        actor=policy_module,
        critic=value_module,
        clip_epsilon=0.2,
        entropy_bonus=True,
        entropy_coef=0.01,
        critic_coef=1.0,
        loss_critic_type="smooth_l1",
    )

In [ ]:
loss_module.eval()

ClipPPOLoss(
  (actor_network_params): TensorDictParams(params=TensorDict(
      fields={
          module: TensorDict(
              fields={
                  0: TensorDict(
                      fields={
                          module: TensorDict(
                              fields={
                                  backbone: TensorDict(
                                      fields={
                                          tcn: TensorDict(
                                              fields={
                                                  0: TensorDict(
                                                      fields={
                                                          conv1: TensorDict(
                                                              fields={
                                                                  bias: Parameter(shape=torch.Size([128]), device=cpu, dtype=torch.float32, is_shared=False),
                                                         

[0.7, 0.3]
Pr:pred_Da:onetwo_L:ppo_BcB:tcn_SeDcat_nb_We:0.7,0.3_Tr:blackbox_Ms:seq
/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/hjkim/RL_TimeSegment/ppo.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via 

In [ ]:
!python main.py --dataset onetwo --seg_dist cat_nb --weights 0.7,0.3 --backbone tcn --loss ppo --target_type blackbox --predictor_type blackbox --mask_type seq

In [ ]:
import ppo
ppo.main(
    backbone='tcn',
    weights=[0.7,0.3],
    seg_dist='cat_nb',
    dataset='onetwo',
    target_type='blackbox', # blackbox, y
    predictor_type='blackbox', # blackbox, predictor
    mask_type='seq', # seq, zero, normal,
)

Pr:blackbox_Da:onetwo_L:ppo_BcB:tcn_SeDcat_nb_We:0.7,0.3_Tr:blackbox_Ms:seq
Epoch : 0
	| Avg Actor Loss: -0.0053 | Avg Critic Loss: 0.0320 
	| Avg Length: 1.7655 | Avg Reward: 0.5508
Best : 0.55
	| Avg Length: 1.0000 | Avg Reward: 0.5507
	| OG  Acc: 1.00  | Masked Acc: 0.37 | OG  F1: 1.00 | Masked F1: 0.25
Epoch : 1
	| Avg Actor Loss: -0.0135 | Avg Critic Loss: 0.0123 
	| Avg Length: 1.7601 | Avg Reward: 0.5561
Best : 0.55
	| Avg Length: 1.0000 | Avg Reward: 0.5522
	| OG  Acc: 1.00  | Masked Acc: 0.37 | OG  F1: 1.00 | Masked F1: 0.24
Epoch : 2


KeyboardInterrupt: 